## Home assignment 05: Bagging and OOB score

Please, fill the lines in the code below.
This is a simplified version of `BaggingRegressor` from `sklearn`. Please, notice, that `sklearn` API is **not preserved**.

Your algorithm should be able to train different instances of the same model class on bootstrapped datasets and to provide [OOB score](https://en.wikipedia.org/wiki/Out-of-bag_error) for the training set.

The model should be passed as model class with no explicit parameters and no parentheses.

Example:
```
import numpy as np
from sklearn.linear_model import LinearRegression

bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
bagging_regressor.fit(LinearRegression, X, y)

```

In [1]:
import numpy as np

In [100]:
class SimplifiedBaggingRegressor:
    def __init__(self, num_bags, oob=False):
        self.num_bags = num_bags
        self.oob = oob
        
    def _generate_splits(self, data: np.ndarray):
        '''
        Generate indices for every bag and store in self.indices_list list
        '''
        self.indices_list = []
        data_length = len(data)
        for bag in range(self.num_bags):
                indices = np.random.randint(0, data_length, data_length)
                self.indices_list.append(indices)
        
    def fit(self, model_constructor, data, target):
        '''
        Fit model on every bag.
        Model constructor with no parameters (and with no ()) is passed to this function.
        
        example:
        
        bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
        bagging_regressor.fit(LinearRegression, X, y)
        '''
        self.data = None
        self.target = None
        self._generate_splits(data)
        assert len(set(list(map(len, self.indices_list)))) == 1, 'All bags should be of the same length!'
        assert list(map(len, self.indices_list))[0] == len(data), 'All bags should contain `len(data)` number of elements!'
        self.models_list = []
        for bag in range(self.num_bags):
            model = model_constructor()
            data_bag, target_bag = data[self.indices_list[bag]], target[self.indices_list[bag]] # Your Code Here
            self.models_list.append(model.fit(data_bag, target_bag)) # store fitted models here
        if self.oob:
            self.data = data
            self.target = target
        
    def predict(self, data):
        '''
        Get average prediction for every object from passed dataset
        '''
        return np.mean(np.array([x.predict(data) for x in self.models_list]), axis=0)
    
    def _get_oob_predictions_from_every_model(self):
        '''
        Generates list of lists, where list i contains predictions for self.data[i] object
        from all models, which have not seen this object during training phase
        '''
        list_of_predictions_lists = [[model.predict(self.data[i].reshape(1, -1)) for j, model in enumerate(self.models_list) if i not in self.indices_list[j]] for i in range(len(self.data))]  
        self.list_of_predictions_lists = np.array(list_of_predictions_lists, dtype=object)
    
    def _get_averaged_oob_predictions(self):
        '''
        Compute average prediction for every object from training set.
        If object has been used in all bags on training phase, return None instead of prediction
        '''
        self._get_oob_predictions_from_every_model()
        print('___TEST___1')
        self.oob_predictions = [np.mean(i) if len(i) != len(self.models_list) else None for i in self.list_of_predictions_lists] # Your Code Here
        print('___TEST___2')
        
        
    def OOB_score(self):
        '''
        Compute mean square error for all objects, which have at least one prediction
        '''
        self._get_averaged_oob_predictions()
        #print(len(self.oob_predictions))
        print(np.array([(self.target[i] - pred)**2 for i, pred in enumerate(self.oob_predictions) if pred is not None]))
        return np.array([(self.target[i] - pred)**2 for i, pred in enumerate(self.oob_predictions) if pred is not None]).mean()

### Local tests:

In [2]:
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm

c:\Projects\ml-course\ml-course-hw\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Simple tests:

In [101]:
for _ in tqdm(range(100)):
    X = np.random.randn(2000, 10)
    y = np.mean(X, axis=1)
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    assert np.mean((predictions - y)**2) < 1e-6, 'Linear dependency should be fitted with almost zero error!'
    assert bagging_regressor.oob, 'OOB feature must be turned on'
    oob_score = bagging_regressor.OOB_score()
    print(oob_score)
    assert oob_score < 1e-6, 'OOB error for linear dependency should be also close to zero!'
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 0.1, 'Probability of missing a bag should be close to theoretical value!'
    
print('Simple tests done!')

  0%|          | 0/100 [00:00<?, ?it/s]c:\Projects\ml-course\ml-course-hw\venv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Projects\ml-course\ml-course-hw\venv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
  0%|          | 0/100 [00:00<?, ?it/s]

___TEST___1
___TEST___2
[2.35926418e-33 1.23259516e-32 5.00771878e-32 ... 4.93038066e-32
 1.10933565e-31 0.00000000e+00]
nan


AssertionError: OOB error for linear dependency should be also close to zero!

#### Medium tests

In [54]:
for _ in tqdm(range(10)):
    X = np.random.randn(200, 150)
    y = np.random.randn(len(X))
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=20, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    average_train_error = np.mean((predictions - y)**2)
    assert bagging_regressor.oob, 'OOB feature must be turned on'
    oob_score = bagging_regressor.OOB_score()
    assert oob_score > average_train_error, 'OOB error must be higher than train error due to overfitting!'
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 0.1, 'Probability of missing a bag should be close to theoretical value!'
    
print('Medium tests done!')

  0%|          | 0/10 [00:18<?, ?it/s]

___TEST___


ValueError: Expected 2D array, got 1D array instead:
array=[ 0.94221337 -1.31426736 -0.93999077  0.8371804   0.78146644 -0.84766328
 -1.60175761  0.35428879  1.5697252   0.31568524 -0.99343914  1.32248738
 -0.29665033 -0.28022558  0.41648008  0.14461974  0.01194447 -1.32871934
 -0.43192878 -1.35665484 -0.03271855 -1.97899938 -0.46833181 -0.44827314
 -0.63787513 -1.49685677 -1.26057531  0.21075326 -0.91733084  0.94847847
 -0.45081447  1.25023298 -0.44454211 -0.35564658  0.98424418  0.95343567
  1.38476089 -0.17780701 -0.35288811 -0.50729358  0.62875137  0.97060981
 -1.69310573  1.19027289  0.17289211  1.71734327  0.56615305  0.49476668
  2.13510913 -0.07636466  0.31053292  1.51526788  1.06811197  1.92947664
 -1.97059253 -0.43253432 -1.73130045  0.87761368 -1.39816388  0.62329479
 -0.12922227 -1.774643   -0.259362    1.13987294  1.09064907  1.20680021
 -2.93108193  1.22849686  1.50986374  1.15266877 -2.05533638  0.68562031
 -0.5792697   0.53092564 -1.51788178  0.40992031 -0.57320292  0.7

#### Complex tests:

In [ ]:
for _ in tqdm(range(10)):
    X = np.random.randn(2000, 15)
    y = np.random.randn(len(X))
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=100, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    oob_score = bagging_regressor.OOB_score()
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 1e-2, 'Probability of missing a bag should be close to theoretical value!'
    
print('Complex tests done!')

In [ ]:
np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)

Great job! Please, save `SimplifiedBaggingRegressor` to  `bagging.py` and submit your solution to the grading system!